<a href="https://colab.research.google.com/github/woodRock/fishy-business/blob/main/code/identification/part/mtgp_runs/R01_S02_Identification_Part_Wrapper_Based_Multi_Tree_GP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multi-tree Genetic Program

## Kicked for inactivity?

To stop a colab notebook from disconnecting, open up the console with CTRL + SHIFT + I, and copy and execute the following code.

```javascript
function ConnectButton(){
    console.log("Connect pushed");
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click()
}
setInterval(ConnectButton,60000);
```

This tricks the browser into thinking the user is active.

In [1]:
!pip install deap
!apt install libgraphviz-dev
!pip install pygraphviz
!pip install skfeature-chappers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 1.4 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libgail-common libgail18 libgtk2.0-0 libgtk2.0-bin libgtk2.0-common libgvc6-plugins-gtk
  librsvg2-common libxdot4
Suggested packages:
  gvfs
The following NEW packages will be installed:
  libgail-common libgail18 libgraphviz-dev libgtk2.0-0 libgtk2.0-bin libgtk2.0-common
  libgvc6-plugins-gtk librsvg2-common libxdot4
0 upgraded, 9 newly installed, 0 to remove and 35 not upgraded.
Need to get 2,433 kB of archives.
After this operation, 7,694 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libgtk2.0-common all 2.24.33-2ubuntu2 [125 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libgtk2.0-0 amd64 2.24.33-2ubuntu2 [2,037 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 libgail18 amd64 

In [2]:
from google.colab import drive
drive.mount('/content/drive')
data_path_gdrive = '/content/drive/MyDrive/AI/Data'
dataset = 'Part' #@param ["Fish", "Part"]

Mounted at /content/drive


## Data

In [3]:
"""
Data - data.py
==============

This is the data module. It contains the functions for loading, preparing, normalizing and encoding the data.
"""

import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
import scipy.io
import os

def encode_labels(y, y_test=None):
    """
    Convert text labels to numbers.

    Args:
        y: The labels.
        y_test: The test labels. Defaults to None.
    """
    le = preprocessing.LabelEncoder()
    y = le.fit_transform(y)
    if y_test is not None:
        y_test = le.transform(y_test)
    return y, y_test, le


def load(filename, folder=''):
    """
    Load the data from the mat file.

    Args:
        filename: The name of the mat file.
        folder: The folder where the mat file is located.
    """
    path = os.path.join(folder, filename)
    mat = scipy.io.loadmat(path)
    return mat


def prepare(mat):
    """
    Load the data from matlab format into memory.

    Args:
        mat: The data in matlab format.
    """
    X = mat['X']
    X = X.astype(float)
    y = mat['Y']
    y = y[:, 0]
    return X,y


def normalize(X_train, X_test):
    """
    Normalize the input features within range [0,1].

    Args:
        X_train: The training data.
        X_test: The test data.
    """
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaler = scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    return X_train, X_test

# Code snippet from elsewhere.

from google.colab import drive
drive.mount('/content/drive')
import os
os.listdir('/content/drive/My Drive')

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

path = ['drive', 'MyDrive', 'AI', 'fish', 'REIMS_data.xlsx']
path = os.path.join(*path)

# Load the dataset
# data = pd.read_excel(path)

print("[INFO] Reading the dataset.")
raw = pd.read_excel(path)

data = raw[~raw['m/z'].str.contains('HM')]
data = data[~data['m/z'].str.contains('QC')]
data = data[~data['m/z'].str.contains('HM')]
X = data.drop('m/z', axis=1) # X contains only the features.
# y = data['m/z'].apply(lambda x:
#                           [1,0,0,0,0,0] if 'Fillet' in x
#                     else ([0,1,0,0,0,0] if 'Heads' in x
#                     else ([0,0,1,0,0,0] if 'Livers' in x
#                     else ([0,0,0,1,0,0] if 'Skins' in x
#                     else ([0,0,0,0,1,0] if 'Guts' in x
#                     else ([0,0,0,0,0,1] if 'Frames' in x
#                     else None ))))))  # Labels for fish parts
y = data['m/z'].apply(lambda x:
                          0 if 'Fillet' in x
                    else  1 if 'Heads' in x
                    else (2 if 'Livers' in x
                    else (3 if 'Skins' in x
                    else (4 if 'Guts' in x
                    else (5 if 'Frames' in x
                    else None )))))  # For fish parts
xs = []
ys = []
for (x,y) in zip(X.to_numpy(),y):
    if y is not None and not np.isnan(y):
       xs.append(x)
       ys.append(y)
X = np.array(xs)
y = np.array(ys)

# file = load(f'{dataset}.mat', folder=data_path_gdrive)
# X,y = prepare(file)
# X,_ = normalize(X,X)
# y, _, le = encode_labels(y)
# labels = le.inverse_transform(np.unique(y))
classes, class_counts = np.unique(y, axis=0, return_counts=True)
n_features = X.shape[1]
n_instances = X.shape[0]
n_classes = len(np.unique(y, axis=0))
class_ratios = np.array(class_counts) / n_instances

print(f"Class Counts: {class_counts}, Class Ratios: {class_ratios}")
print(f"Number of features: {n_features}\nNumber of instances: {n_instances}\nNumber of classes {n_classes}.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[INFO] Reading the dataset.
Class Counts: [6 6 3 6 6 3], Class Ratios: [0.2 0.2 0.1 0.2 0.2 0.1]
Number of features: 1023
Number of instances: 30
Number of classes 6.


## Activation Function

## Operators

In [4]:
import math
import copy
import random
import operator
from re import I
from operator import attrgetter
from functools import wraps, partial

import numpy as np

from deap import algorithms
from deap.algorithms import varAnd
from deap import base, creator, tools, gp
from deap.gp import PrimitiveTree, Primitive, Terminal

from sklearn.metrics import balanced_accuracy_score

pset = gp.PrimitiveSet("MAIN", n_features)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(operator.neg, 1)
# pset.addEphemeralConstant("rand101", lambda: random.randint(-1,1))

## Fitness Function

In [5]:
toolbox = base.Toolbox()

minimized = False
if minimized:
    weight = -1.0
else:
    weight = 1.0

weights = (weight,)

if minimized:
    creator.create("FitnessMin", base.Fitness, weights=weights)
    creator.create("Individual", list, fitness=creator.FitnessMin)
else:
    creator.create("FitnessMax", base.Fitness, weights=weights)
    creator.create("Individual", list, fitness=creator.FitnessMax)

def quick_evaluate(expr: PrimitiveTree, pset, data, prefix='ARG'):
    """ Quick evaluate offers a 500% speedup for the evluation of GP trees.

    The default implementation of gp.compile provided by the DEAP library is
    horrendously inefficient. (Zhang 2022) has shared his code which leads to a
    5x speedup in the compilation and evaluation of GP trees when compared to the
    standard library approach.

    For multi-tree GP, this speedup factor is invaluable! As each individual conists
    of m trees. For the fish dataset we have 4 classes, each with 3 constructed features,
    which corresponds to 4 classes x 3 features = 12 trees for each individual.
    12 trees x 500% speedup = 6,000% overall speedup, or 60 times faster.
    The 500% speedup is fundamental, for efficient evaluation of multi-tree GP.

    Args:
        expr (PrimitiveTree): The uncompiled (gp.PrimitiveTree) GP tree.
        pset: The primitive set.
        data: The dataset to evaluate the GP tree for.
        prefix: Prefix for variable arguments. Defaults to ARG.

    Returns:
        The (array-like) result of the GP tree evaluate on the dataset .
    """
    result = None
    stack = []
    for node in expr:
        stack.append((node, []))
        while len(stack[-1][1]) == stack[-1][0].arity:
            prim, args = stack.pop()
            if isinstance(prim, Primitive):
                result = pset.context[prim.name](*args)
            elif isinstance(prim, Terminal):
                if prefix in prim.name:
                    result = data[:, int(prim.name.replace(prefix, ''))]
                else:
                    result = prim.value
            else:
                raise Exception
            if len(stack) == 0:
                break # If stack is empty, all nodes should have been seen
            stack[-1][1].append(result)
    return result

def compileMultiTree(expr, pset):
    """Compile the expression represented by a list of trees.

    A variation of the gp.compileADF method, that handles Multi-tree GP.

    Args:
        expr: Expression to compile. It can either be a PrimitiveTree,
                 a string of Python code or any object that when
                 converted into string produced a valid Python code
                 expression.
        pset: Primitive Set

    Returns:
        A set of functions that correspond for each tree in the Multi-tree.
    """
    funcs = []
    gp_tree = None
    func = None

    for subexpr in expr:
        gp_tree = gp.PrimitiveTree(subexpr)
        # 5x speedup by manually parsing GP tree (Zhang 2022) https://mail.google.com/mail/u/0/#inbox/FMfcgzGqQmQthcqPCCNmstgLZlKGXvbc
        func = quick_evaluate(gp_tree, pset, X, prefix='ARG')
        funcs.append(func)

    # Hengzhe's method returns the features in the wrong rotation for multi-tree
    features = np.array(funcs).T
    return features

# MCIFC constructs 8 feautres for a (c=4) multi-class classification problem (Tran 2019).
# c - number of classes, r - construction ratio, m - total number of constructed features.
# m = r * c = 2 ratio * 4 classes = 8 features

r = 3
c = n_classes
m = r * c

toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.expr, n=m)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", compileMultiTree)

## Infinite speedup!?

For multi-tree GP, this speedup factor is invaluable! As each individual conists of m trees. For the fish dataset we have 4 classes, each with 3 constructed features, which corresponds to 4 classes x 3 features = 12 trees for each individual. 12 trees x 500% speedup = 6,000% overall speedup, or 60 times faster. The 500% speedup is fundamental, for efficient evaluation of multi-tree GP.

The evaluation below shows my calculations may still be wrong. And perhaps, it is even faster:

In [6]:
first = toolbox.population(n=1)[0]

subtree = first[0]
gp_tree = gp.PrimitiveTree(subtree)

%timeit gp.compile(gp_tree, pset)
%timeit quick_evaluate(gp_tree, pset, X)

3.96 ms ± 1.24 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
2.78 µs ± 1.29 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [7]:
import numpy as np
from sklearn.model_selection import StratifiedKFold, train_test_split


X,_ = normalize(X,X)
train_split = 0.8
X_train, X_temp, y_train, y_temp = train_test_split(X, y, stratify=y, test_size=(1-train_split), random_state=1998)
# X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, stratify=y_temp, test_size=0.5, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=1998)

# classes, class_counts = np.unique(y_train, axis=0, return_counts=True)
# n_features = X.shape[1]
# n_instances = X.shape[0]
# n_classes = len(np.unique(y_train, axis=0))
# class_ratios = np.array(class_counts) / n_instances
# print(f"Class Counts: {class_counts}, Class Ratios: {class_ratios}")
# print(f"Number of features: {n_features}\nNumber of instances: {n_instances}\nNumber of classes {n_classes}.")

def is_same_class(a,b):
    # print(f"[DEBUG] {a[1], b[1]}")
    return a[1] == b[1]

# Euclidean distance
def euclidian_distance(a,b):
    a,b = a[0], b[0]
    return np.linalg.norm(a-b)

def intraclass_distance(_X,_y):
    data = list(zip(_X, _y))
    length = len([1 if is_same_class(a,b) else 0 for idx, a in enumerate(data) for b in data[idx + 1:]])
    print(f"[DEBUG] length: {length}")
    pair_length = sum([1 if is_same_class(a,b) else 0 for idx, a in enumerate(data) for b in data[idx + 1:]])
    print(f"[DEBUG] pair length: {pair_length}")
    return sum([euclidian_distance(a,b) if is_same_class(a,b) else 0 for idx, a in enumerate(data) for b in data[idx + 1:]]) / (pair_length * X.shape[1])

def interclass_distance(_X,_y):
    data = list(zip(_X, _y))
    length = len([1 if not is_same_class(a,b) else 0 for idx, a in enumerate(data) for b in data[idx + 1:]])
    print(f"[DEBUG] length: {length}")
    pair_length = sum([1 if not is_same_class(a,b) else 0 for idx, a in enumerate(data) for b in data[idx + 1:]])
    print(f"[DEBUG] pair length: {pair_length}")
    return sum([euclidian_distance(a,b) if not is_same_class(a,b) else 0 for idx, a in enumerate(data) for b in data[idx + 1:]]) / (pair_length * X.shape[1])

for i in range(1):
    b = intraclass_distance(X_train, y_train)
    print(f"[DEBUG] intraclass distance: {b}")
    c = interclass_distance(X_train, y_train)
    print(f"[DEBUG] interclass distance: {c}")

[DEBUG] length: 276
[DEBUG] pair length: 42
[DEBUG] intraclass distance: 0.008907155425521254
[DEBUG] length: 276
[DEBUG] pair length: 234
[DEBUG] interclass distance: 0.011729963180235807


In [8]:
from sklearn.svm import LinearSVC as svm
from sklearn.model_selection import StratifiedKFold, train_test_split

def is_same_class(a,b):
    return a[1] == b[1]

# Euclidean distance
def euclidian_distance(a,b):
    a,b = a[0], b[0]
    return np.linalg.norm(a-b)

def intraclass_distance(_X,_y):
    data = list(zip(_X, _y))
    pair_length = sum([1 if is_same_class(a,b) else 0 for idx, a in enumerate(data) for b in data[idx + 1:]])
    d = sum([euclidian_distance(a,b) if is_same_class(a,b) else 0 for idx, a in enumerate(data) for b in data[idx + 1:]]) / (pair_length * X.shape[1])
    return d

def interclass_distance(_X,_y):
    data = list(zip(_X, _y))
    pair_length = sum([1 if not is_same_class(a,b) else 0 for idx, a in enumerate(data) for b in data[idx + 1:]])
    d = sum([euclidian_distance(a,b) if not is_same_class(a,b) else 0 for idx, a in enumerate(data) for b in data[idx + 1:]]) / (pair_length * X.shape[1])
    return d

def wrapper_classification_accuracy(X, k=10, verbose=False):
    """ Evaluate balanced classification accuracy over stratified k-fold cross validation.

    This method is our fitness measure for an individual. We measure each individual
    based on its balanced classification accuracy using 10-fold cross-validation on
    the training set.

    If verbose, we evaluate performance on the test set as well, and print the results
    to the standard output. By default, only the train set is evaluated, which
    corresponds to a 2x speedup for training, when compared to the verbose method.

    Args:
        X: entire dataset, train and test.
        k: Number of folds, for cross validation. Defaults to 10.
        verbose: If true, prints stuff. Defaults to false.

    Returns:
        Average balanced classification accuracy with 10-fold CV on training set.
    """

    # X_train, X_temp, y_train, y_temp = train_test_split(X, y, stratify=y, test_size=0.7, random_state=42)
    # X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, stratify=y_temp, test_size=0.5, random_state=42)

    train_split = 0.66
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, stratify=y, test_size=(1-train_split), random_state=1998)
    # X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, stratify=y_temp, test_size=0.5, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=1998)

    # train_accs = []
    # test_accs = []
    # skf = StratifiedKFold(n_splits=3)
    # for train_idx, test_idx in skf.split(X,y):
        # X_train, X_test = X[train_idx], X[test_idx]
        # y_train, y_test = y[train_idx], y[test_idx]

        # Convergence errors for the fish part dataset.
        # Need to use a different SVM hyperparameters for this dataset.
        # model = svm(penalty='l2', max_iter=10_000)
    model = svm()
    model.fit(X_train, y_train)
    y_predict = model.predict(X_train)
    train_acc = balanced_accuracy_score(y_train, y_predict)
    # train_accs.append(train_acc)
    # test_accs.append(test_acc)

    y_predict = model.predict(X_val)
    val_acc = balanced_accuracy_score(y_val, y_predict)

    if verbose:
        # Only evaluate test set if verbose!
        # Results in 2x speedup for training.
        y_predict = model.predict(X_test)
        test_acc = balanced_accuracy_score(y_test, y_predict)
        print(f"Train accuracy: {train_acc}, Validation accuracy: {val_acc}, Test accuracy: {test_acc}")

    # Distance-based regularization method, intra/inter class distance.
    train_intraclass_distance = intraclass_distance(X_train, y_train)
    train_interclass_distance = interclass_distance(X_train, y_train)
    val_intraclass_distance = intraclass_distance(X_val, y_val)
    val_interclass_distance = interclass_distance(X_val, y_val)

    if verbose:
        print(f"Train intra-class: {train_intraclass_distance}, Val intra-class: {val_intraclass_distance}")
        print(f"Train inter-class: {train_interclass_distance}, Val inter-class: {val_interclass_distance}")

    alpha = 0.5
    beta = 0.25
    gamma = 0.8
    accuracy = alpha * train_acc + (1 - alpha) * val_acc
    train_distance = beta * (1 - train_intraclass_distance) + beta * train_interclass_distance
    val_distance = beta * (1 - val_intraclass_distance) + beta * val_interclass_distance
    total_distance = train_distance + val_distance
    fitness = gamma * accuracy + (1 - gamma) * total_distance
    # Fitness value must be a tuple.
    assert fitness <= 1, f"fitness {fitness} should be normalized, and cannot exceed 1"
    if fitness > 1:
        print(f"Train intra-class: {train_intraclass_distance}, Val intra-class: {val_intraclass_distance}")
        print(f"Train inter-class: {train_interclass_distance}, Val inter-class: {val_interclass_distance}")
    return fitness

wrapper_classification_accuracy(X, verbose=True)

Train accuracy: 1.0, Validation accuracy: 0.625, Test accuracy: 0.5
Train intra-class: 0.009351435900701907, Val intra-class: 0.008435299933447275
Train inter-class: 0.011964009297661593, Val inter-class: 0.010493423399204789


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


0.7502335348431358

In [9]:
#  DEBUG : REMOVE THIS !!!
import warnings
warnings.filterwarnings('ignore')

def xmate(ind1, ind2):
    """ Reproduction operator for multi-tree GP, where trees are represented as a list.

    Crossover happens to a subtree that is selected at random.
    Crossover operations are limited to parents from the same tree.

    FIXME: Have to compile the trees (manually), which is frustrating.

    Args:
        ind1 (Individual): The first parent.
        ind2 (Individual): The second parent

    Returns:
        ind1, ind2 (Individual, Individual): The children from the parents reproduction.
    """
    n = range(len(ind1))
    selected_tree_idx = random.choice(n)
    for tree_idx in n:
        g1, g2 = gp.PrimitiveTree(ind1[tree_idx]), gp.PrimitiveTree(ind2[tree_idx])
        if tree_idx == selected_tree_idx:
            ind1[tree_idx], ind2[tree_idx] = gp.cxOnePoint(g1, g2)
        else:
            ind1[tree_idx], ind2[tree_idx] = g1, g2
    return ind1, ind2


def xmut(ind, expr):
    """ Mutation operator for multi-tree GP, where trees are represented as a list.

    Mutation happens to a tree selected at random, when an individual is selected for crossover.

    FIXME: Have to compile the trees (manually), which is frustrating.

    Args:
        ind: The individual, a list of GP trees.
    """
    n = range(len(ind))
    selected_tree_idx = random.choice(n)
    for tree_idx in n:
        g1 = gp.PrimitiveTree(ind[tree_idx])
        if tree_idx == selected_tree_idx:
            indx = gp.mutUniform(g1, expr, pset)
            ind[tree_idx] = indx[0]
        else:
            ind[tree_idx] = g1
    return ind,


def evaluate_classification(individual, alpha = 0.9, verbose=False):
    """
    Evalautes the fitness of an individual for multi-tree GP multi-class classification.

    We maxmimize the fitness when we evaluate the accuracy + regularization term.

    Args:
        individual (Individual): A candidate solution to be evaluated.
        alpha (float): A parameter that balances the accuracy and regularization term. Defaults to 0.98.

    Returns:
        accuracy (tuple): The fitness of the individual.
    """
    features = toolbox.compile(expr=individual, pset=pset)
    fitness = wrapper_classification_accuracy(features, verbose=verbose)
    return fitness,


toolbox.register('evaluate', evaluate_classification)
toolbox.register("select", tools.selTournament, tournsize=7)
toolbox.register("mate", xmate)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", xmut, expr=toolbox.expr_mut)


def staticLimit(key, max_value):
    """
    A variation of gp.staticLimit that works for Multi-tree representation.
    This works for our altered xmut and xmate genetic operators for mutli-tree GP.
    If tree depth limit is exceeded, the genetic operator is reverted.

    When an invalid (over the limit) child is generated,
    it is simply replaced by one of its parents, randomly selected.

    Args:
        key: The function to use in order the get the wanted value. For
             instance, on a GP tree, ``operator.attrgetter('height')`` may
             be used to set a depth limit, and ``len`` to set a size limit.
        max_value: The maximum value allowed for the given measurement.
             Defaults to 17, the suggested value in (Koza 1992)

    Returns:
        A decorator that can be applied to a GP operator using \
        :func:`~deap.base.Toolbox.decorate`

    References:
        1. Koza, J. R. G. P. (1992). On the programming of computers by means
            of natural selection. Genetic programming.
    """

    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            keep_inds = [[copy.deepcopy(tree) for tree in ind] for ind in args]
            new_inds = list(func(*args, **kwargs))
            for ind_idx, ind in enumerate(new_inds):
                for tree_idx, tree in enumerate(ind):
                    if key(tree) > max_value:
                        random_parent = random.choice(keep_inds)
                        new_inds[ind_idx][tree_idx] = random_parent[tree_idx]
            return new_inds
        return wrapper
    return decorator

# See https://groups.google.com/g/deap-users/c/pWzR_q7mKJ0
toolbox.decorate("mate", staticLimit(key=operator.attrgetter("height"), max_value=8))
toolbox.decorate("mutate", staticLimit(key=operator.attrgetter("height"), max_value=8))


def SimpleGPWithElitism(population, toolbox, cxpb, mutpb, ngen, stats=None,
             halloffame=None, verbose=__debug__):
    """
    Elitism for Multi-Tree GP for Multi-Class classification.
    A variation of the eaSimple method from the DEAP library that supports

    Elitism ensures the best individuals (the elite) from each generation are
    carried onto the next without alteration. This ensures the quality of the
    best solution monotonically increases over time.
    """
    logbook = tools.Logbook()
    logbook.header = ['gen', 'nevals'] + (stats.fields if stats else [])

    invalid_ind = [ind for ind in population if not ind.fitness.valid]
    fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)

    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    if halloffame is None:
        raise ValueError("halloffame parameter must not be empty!")

    halloffame.update(population)
    hof_size = len(halloffame.items) if halloffame.items else 0

    record = stats.compile(population) if stats else {}
    logbook.record(gen=0, nevals=len(invalid_ind), **record)

    if verbose:
        print(logbook.stream)

    for gen in range(1, ngen + 1):
        offspring = toolbox.select(population, len(population) - hof_size)
        offspring = algorithms.varAnd(offspring, toolbox, cxpb, mutpb)
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)

        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        offspring.extend(halloffame.items)
        halloffame.update(offspring)
        population[:] = offspring

        record = stats.compile(population) if stats else {}
        logbook.record(gen=gen, nevals=len(invalid_ind), **record)

        if verbose:
            print(logbook.stream)

    return population, logbook


def train(generations=100, population=100, elitism=0.1, crossover_rate=0.5, mutation_rate=0.1):
    """
    This is a Multi-tree GP with Elitism for Multi-class classification.

    Args:
        generations: The number of generations to evolve the populaiton for.
        elitism: The ratio of elites to be kept between generations.
        crossover_rate: The probability of a crossover between two individuals.
        mutation_rate: The probability of a random mutation within an individual.

    Returns:
        pop: The final population the algorithm has evolved.
        log: The logbook which can record important statistics.
        hof: The hall of fame contains the best individual solutions.
    """
    random.seed(420)
    pop = toolbox.population(n=population)

    mu = round(elitism * population)
    if elitism > 0:
        # See https://www.programcreek.com/python/example/107757/deap.tools.HallOfFame
        hof = tools.HallOfFame(mu)
    else:
        hof = None

    stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
    length = lambda a: np.max(list(map(len, a)))
    stats_size = tools.Statistics(length)

    mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
    mstats.register("avg", np.mean)
    mstats.register("std", np.std)
    mstats.register("min", np.min)
    mstats.register("max", np.max)

    pop, log = SimpleGPWithElitism(pop, toolbox, crossover_rate, mutation_rate,
                                   generations, stats=mstats, halloffame=hof,
                                   verbose=True)
    return pop, log, hof


"""
DeJong (1975), p=50-100, m=0.001, c=0.6
Grefenstette (1986), p=30, m=0.01, c=0.95
Schaffer et al., (1989), p=20-30, m=0.005-0.01, c=0.75-0.95

References:
    1. Patil, V. P., & Pawar, D. D. (2015). The optimal crossover or mutation
    rates in genetic algorithm: a review. International Journal of Applied
    Engineering and Technology, 5(3), 38-41.
"""

beta = 1
population = n_features * beta
generations = 200
elitism = 0.1
crossover_rate = 0.8
mutation_rate = 0.2

assert crossover_rate + mutation_rate == 1, "Crossover and mutation sums to 1 (to please the Gods!)"

pop, log, hof = train(generations, population, elitism, crossover_rate, mutation_rate)

   	      	                                 fitness                                 	                      size                      
   	      	-------------------------------------------------------------------------	------------------------------------------------
gen	nevals	avg     	gen	max     	min     	nevals	std      	avg    	gen	max	min	nevals	std     
0  	1023  	0.606989	0  	0.900033	0.433351	1023  	0.0827613	6.79081	0  	7  	3  	1023  	0.592585
1  	758   	0.712961	1  	0.900034	0.483353	758   	0.0754621	6.86804	1  	13 	3  	758   	0.811203
2  	758   	0.787039	2  	0.900072	0.516689	758   	0.0718059	6.91007	2  	13 	4  	758   	0.903846
3  	786   	0.837735	3  	0.900072	0.6167  	786   	0.0562504	6.90127	3  	13 	4  	786   	0.721834
4  	773   	0.851711	4  	0.900077	0.483364	773   	0.0608073	6.9306 	4  	13 	6  	773   	0.762047
5  	752   	0.870983	5  	0.900087	0.616749	752   	0.0514452	7.02444	5  	12 	5  	752   	0.702174
6  	742   	0.882327	6  	0.900091	0.600061	742   	0.0402569	7.1828 	

In [10]:
for i in range(len(hof)):
    evaluate_classification(hof[i], verbose=True)

Train accuracy: 0.9166666666666666, Validation accuracy: 1.0, Test accuracy: 0.625
Train intra-class: 0.019487521171979768, Val intra-class: 0.00036419212840000943
Train inter-class: 0.05419066222948262, Val inter-class: 0.07299579286552665
Train accuracy: 1.0, Validation accuracy: 1.0, Test accuracy: 0.625
Train intra-class: 0.019306967088907105, Val intra-class: 0.00022001105249983916
Train inter-class: 0.05376843394200691, Val inter-class: 0.07245779371698427
Train accuracy: 1.0, Validation accuracy: 0.75, Test accuracy: 0.375
Train intra-class: 0.019346494124887585, Val intra-class: 0.0002655799120009938
Train inter-class: 0.053809815766018905, Val inter-class: 0.07246823450262226
Train accuracy: 0.9166666666666666, Validation accuracy: 1.0, Test accuracy: 0.625
Train intra-class: 0.019342873578475, Val intra-class: 0.00026106690184419205
Train inter-class: 0.05379708802153428, Val inter-class: 0.07246972129604322
Train accuracy: 0.875, Validation accuracy: 1.0, Test accuracy: 0.62

In [11]:
evaluate_classification(hof[0], verbose=True)

Train accuracy: 1.0, Validation accuracy: 1.0, Test accuracy: 0.625
Train intra-class: 0.019487521171979768, Val intra-class: 0.00036419212840000943
Train inter-class: 0.05419066222948262, Val inter-class: 0.07299579286552665


(0.9053667370897315,)

In [12]:
train_split = 0.66
X_train, X_temp, y_train, y_temp = train_test_split(X, y, stratify=y, test_size=(1-train_split), random_state=42)
# X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, stratify=y_temp, test_size=0.5, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


# print(f"np.unique(y_train): {np.unique(y_train)}\n np.unique(y_val): {np.unique(y_val)} \n np.unique(y_test): {np.unique(y_test)}")
for (name, ds) in [("train", y_train), ("val", y_val), ("test", y_test)]:
    print(f"{name}: {np.unique(ds)}")

    classes, class_counts = np.unique(ds, axis=0, return_counts=True)
    n_features = X.shape[1]
    n_instances = X.shape[0]
    n_classes = len(np.unique(ds, axis=0))
    class_ratios = np.array(class_counts) / n_instances

    print(f"Class Counts: {class_counts}, Class Ratios: {class_ratios}")
    print(f"Number of features: {n_features}\nNumber of instances: {n_instances}\nNumber of classes {n_classes}.")

train: [0. 1. 2. 3. 4. 5.]
Class Counts: [4 4 2 4 3 2], Class Ratios: [0.13333333 0.13333333 0.06666667 0.13333333 0.1        0.06666667]
Number of features: 1023
Number of instances: 30
Number of classes 6.
val: [0. 2. 3. 4.]
Class Counts: [2 1 1 1], Class Ratios: [0.06666667 0.03333333 0.03333333 0.03333333]
Number of features: 1023
Number of instances: 30
Number of classes 4.
test: [1. 3. 4. 5.]
Class Counts: [2 1 2 1], Class Ratios: [0.06666667 0.03333333 0.06666667 0.03333333]
Number of features: 1023
Number of instances: 30
Number of classes 4.


## Visualization

In [13]:
from deap import base, creator, gp
import pygraphviz as pgv

multi_tree = hof[0]
for t_idx,tree in enumerate(multi_tree):
    nodes, edges, labels = gp.graph(tree)

    g = pgv.AGraph()
    g.add_nodes_from(nodes)
    g.add_edges_from(edges)
    g.layout(prog="dot")

    for i in nodes:
        n = g.get_node(i)
        n.attr["label"] = labels[i]

    g.draw(f"tree-{t_idx}.pdf")

## Changelog

| Date | Title | Description | Update |
| --- | --- | --- | ---- |
| 2024-02-27 16:31 | Fix fitness | Fixed fitness function to penalize intra-class distance <br> and reward inter-class distance. |
| 2024-02-26 15:59 | Intra/inter | Updated fitness value to include a normalized sum of <br> intraclass and interclass distance |
| 2024-02-14 16:56 | Mass spectra | Applications for MT-GP for rapid mass spectrometry dataset. | |
| 2022-08-21 17:30 | Multi-Objective - Onehot Encoding | Change to multi-objective problem, one-vs-all with a tree classifier for each class.<br> Y labels are encoded in onehot encodings, error is absolute difference between $|\hat{y} - y|$|
| 2022-08-22 20:44 | Non-linearity |  Introduce $ round . sigmoid $ to evaluate_classification() method.<br>Previously, we push each tree to predict either a 0 or 1 value with the onehot encoding representation.<br>Now, the non-linearity will map any negative value to a negative class 0, and any positive value to positive class 1.|
| 2022-08-22 21:06 | ~~Genetic operators for tree with worst fitness~~ | Only apply the genetic operators, crossover and mutation, to the tree with the worst fitness.<br> This guarantees monotonic improvement for the Multi-tree between generations, the best performing tree remain unaltered.| (Update) This was very slow, and inefficient,<br> basically turned the GP into a single objective,<br>that balances multi-objective fitness functions. |
| 2022-08-22 21:15 | Halloffame Equality Operator | Numpy equality function (operators.eq) between two arrays returns the equality element wise,<br>which raises an exception in the if similar() check of the hall of fame. <br> Using a different equality function like numpy.array_equal or numpy.allclose solve this issue.|
| 2022-08-22 23:22 | Elitism as aggregate best tree | Perform elitsim by constructing the best tree, as the tree with best fitness from each clas.<br>The goal is to have monotonous improvement across the multiple objective functions.|
| 2022-08-22 23:32 | Update fitness for elite | The elitism was not working as intended, as the multi-objectives didn't appear to increase monotnously.<br> This was because the aggregate fitness was not being assigned to the best individual after it was created.<br>Therefore the best invidiual was not passed on to the next generation. |
| 2022-08-22 02:28 | staticLimit max height | Rewrite the gp.staticLimit decorator function to handle the Multi-tree representation.<br>Note: size and depth are different values!<br>depth is the maximum length of a root-to-leaf traversal,<br>size is the total number of nodes.|
| 2022-08-24 9:37 | Evaluate Mutli-tree train accuracy | Take the classification accuracy as the argmax for the aggregate multitree.<br> 74% training accuracy, which is not ideal, but this shall improve with time.|
| 2022-08-25 13:30 | Single-objective fitness | Change the fitness function to a single objective fitness function.<br>This forces the multi-tree GP to find the best tree subset for one-vs-rest classification performance.|
| 2022-08-25 20:01 | Fitness = Balanced accuracy + distance measure | Implement the fitness function for MCIFC, but for multi-class classification from (Tran 2019) |
| 2022-08-26 21:27 | Sklearn Balanced Accuracy | Changed to the balanced accuracy metric from sklearn.<br>This is much easier to use for now, probably faster than the previous method as well. |
| 2022-09-05 17:00 | Reject invalid predictions | Change the fitness function to reject invalid predictioctions outright -<br>e.g. multi-label or zero-label predictions<br>- when computing the balanced accuracy for the fitness function. |
| 2022-09-13 19:00 | Mutation + Crossover = 100% | Ensure the mutation and crossover rate sum to 100%,<br>not necessary with deap, but good to avoid conference questions |
| 2022-09-13 21:00 | Feature Construction | Changed to Wrapper-based Feature Construction with Multi-tree GP.|
| 2022-09-13 21:34 | $m = r \times c$ | Add more trees, following example from (Tran 2019).<br> With 8 trees for a multi-class classification<br> $m = r \times c = 8$ trees, where number of classes $c = 4$, and reconstruction ratio $r = 2$/ |
| 2022-09-30 19:49 | Quick Evalaute | Manually parse the GP trees, 5x speedup for DEAP in Python (Zhang 2022). |
| 2022-10-13 6:02 | Ignore timestamps after 4500 | Ignore timestamps after 4500 did not improve accuracy for SVM classifier.<br>So the bizzare pattern that occurs on GC-MS image there has important information.<br> Should investigate this further, perhaps ask Daniel as he is a domain expert.|
| 2022-10-13 22:16 | Cross validation | Evaluate the mean balanced classification accuracy over stratified k-fold cross validation.|
| 2023-01-13 20:58 | 2x speedup | Only evaluate test set for verbose alternative of the evaluate_classification method.<br> This results in a 2x speedup in the efficiency of the training regime.|